<a href="https://colab.research.google.com/github/Victor-Ribeiro-Acosta/Projetos-DIO-Python-Developer/blob/main/Atividades/Atividades_com_frameworks/Projeto_Aplica%C3%A7%C3%A3o_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Desafio 1: integrando ao banco relacional SQLite

In [ ]:
!pip install sqlalchemy

In [ ]:

from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import Session

from sqlalchemy import Column
from sqlalchemy import String
from sqlalchemy import Integer
from sqlalchemy import Float
from sqlalchemy import ForeignKey
from sqlalchemy import select

from sqlalchemy import create_engine


from random import randint


base = declarative_base()

class Cliente(base):

  __tablename__ = 'Cliente'

  id = Column(Integer, primary_key = True, autoincrement = True)
  nome = Column(String)
  cpf = Column(String(11))
  endereco = Column(String)

  conta = relationship('Conta', back_populates = 'cliente', cascade = 'all, delete-orphan')

  def __repr__(self):
    return f"Cliente(id - {self.id}, nome - {self.nome}, cpf - {self.cpf}, endereço - {self.endereco})"


class Conta(base):

  __tablename__ = "Contas_Bancarias"

  id = Column(Integer, primary_key = True, autoincrement = True)
  id_cliente = Column(Integer, ForeignKey('Cliente.id'))
  numero = Column(String)
  agencia = Column(String)
  tipo = Column(String)
  saldo = Column(Float)

  cliente = relationship('Cliente', back_populates = 'conta')

  def __repr__(self):
    return f"Conta(número - {self.numero}, agência - {self.agencia}, tipo - {self.tipo}, saldo - {self.saldo})"


engine = create_engine("sqlite:///:Banco_dados")
base.metadata.create_all(engine)

######################################################################

def criar_numero_conta():

  lista_numeros = []
  for numero in range(8):
    if numero == 6:
      lista_numeros.append('-')
    else:
      lista_numeros.append(str(randint(0,9)))

  return ''.join(lista_numeros)


agencias = {'São Paulo': '4456-1', 'Campinas': '4456-2', 'Piracicapa': '4456-3',
            'Porto Alegre': '4455-1', 'Caxias do Sul': '4455-2',
            'Rio de Janeiro': '4457-1', 'Cabo Frio':'4457-1'}
######################################################################

menu = """
[1] - Cadastrar cliente
[2] - Consultar
[3] - sair

informe sua ação desejada
"""

################################################################

        # Sistema Principal

operacao = int(input(menu))

if operacao == 1:

  nome = str(input('Digite seu nome completo: '))
  cpf = str(input('Digite seu cpf: '))
  endereco = str(input('Digite seu endereço completo: '))
  tipo_conta = str(input('Informe o tipo de conta [corrente ou poupança]: '))
  cidade = str(input('Digite sua cidade: '))

  if cidade not in agencias.keys():
    print('Não há agencias disponíveis na cidade indicada.')

  else:

    agencia = agencias[cidade]

    with Session(engine) as sessao:
      cliente_banco = Cliente(
          nome = nome,
          cpf = cpf,
          endereco = endereco,

          conta = Conta(numero = criar_numero_conta(),
                        agencia = agencia,
                        tipo = tipo_conta,
                        saldo = 1000.0)
        )

      sessao.add_all([cliente_banco])
      sessao.commit()
      sessao.close()

elif operacao == 2:

  cpf = str('Informe seu cpf')

  with Session(engine) as sessao:
    consulta = select(Cliente.nome, Cliente.cpf, Conta.numero, Conta.agencia, Conta.tipo).join_from(Cliente, Conta).where(Conta.cpf.in_(cpf))
    for resultados in sessao.scalars(consulta):
      print(resultados)
    sessao.close()
